In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [1]:
pip list

Package                   Version
------------------------- -----------
absl-py                   2.3.1
altair                    5.5.0
asttokens                 3.0.0
astunparse                1.6.3
attrs                     25.3.0
beautifulsoup4            4.13.4
blinker                   1.9.0
bs4                       0.0.2
cachetools                6.1.0
certifi                   2025.7.9
charset-normalizer        3.4.2
click                     8.2.1
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.2
cycler                    0.12.1
debugpy                   1.8.14
decorator                 5.2.1
et_xmlfile                2.0.0
executing                 2.2.0
fastjsonschema            2.21.1
Flask                     3.1.1
flatbuffers               25.2.10
fonttools                 4.58.4
gast                      0.6.0
gensim                    4.3.3
gitdb                     4.0.12
GitPython                 3.1.44
google-pasta       

In [2]:
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo1=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender1 = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler1 = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [9]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo1.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo1.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Users\Administrator\Downloads\Udemy\python\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [12]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender1.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [13]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
## Scaling the input data
input_scaled=scaler1.transform(input_df)
input_scaled#ye sab 1 wale pickle file wale hai

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step


array([[0.03245578]], dtype=float32)

In [17]:
prediction_proba = prediction[0][0]

In [18]:
prediction_proba

0.032455776

In [19]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
